# The N-Queens Problem

The following works gives a simple solver for the [N-Queens Problem](https://en.wikipedia.org/wiki/Eight_queens_puzzle). We represent the board as a [constraint satisfaction problem](https://en.wikipedia.org/wiki/Constraint_satisfaction_problem) and find a valid solution using the [backtracking algorithm](https://en.wikipedia.org/wiki/Backtracking). That no two queens can be in the same diagonal, row or column are the constraints to enforce. We represent these constraints in [SymPy](http://www.sympy.org/en/index.html), a light-weight computer algebra system written in Python.

The backtracking algorithm retains a partial set of assignments of queens to squares, checking, at each assignment, whether all stipulated constraints hold. If so, it continues in recursive fashion, initiating the next step with the current assignment set. If not, it "backtracks:" the most recent assignment is undone, and the algorithm explores other assignments in turn. The backtracking algorithm can be further improved by strategies like forward-checking, intelligent value-ordering, etc. These are not implemented here.

Finally, this works builds off of code from [Udacity's Artificial Intelligence Nanodegree](https://classroom.udacity.com/nanodegrees/nd889).

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt

from util import constraint, displayBoard
from sympy import *
from IPython.display import display
init_printing()
%matplotlib inline

## Declare variables

In [6]:
first_queen_row = symbols('first\_queen\_row')
second_queen_row = symbols('second\_queen\_row')
first_queen_col = symbols('first\_queen\_col')
second_queen_col = symbols('second\_queen\_col')

## Declare constraints

In [7]:
diffRow = constraint('diffRow', Ne(first_queen_row, second_queen_row))
diffDiag = constraint('diffDiag', Ne( abs(first_queen_row - second_queen_row), abs(first_queen_col - second_queen_col) ))

## Test constraints

In [8]:
# Test diffRow and diffDiag
_x = symbols("x:3")

# generate a diffRow instance for testing
diffRow_test = diffRow.subs({first_queen_row: _x[0], second_queen_row: _x[1]})

assert(len(diffRow_test.free_symbols) == 2)
assert(diffRow_test.subs({_x[0]: 0, _x[1]: 1}) == True)
assert(diffRow_test.subs({_x[0]: 0, _x[1]: 0}) == False)
assert(diffRow_test.subs({_x[0]: 0}) != False)  # partial assignment is not false
print("Passed all diffRow tests.")

# generate a diffDiag instance for testing
diffDiag_test = diffDiag.subs({first_queen_row: 1, second_queen_row: 3, first_queen_col: _x[0], second_queen_col: _x[2]})

assert(len(diffDiag_test.free_symbols) == 2)
assert(diffDiag_test.subs({_x[0]: 0, _x[2]: 2}) == False)
assert(diffDiag_test.subs({_x[0]: 0, _x[2]: 0}) == True)
assert(diffDiag_test.subs({_x[0]: 0}) != False)  # partial assignment is not false
print("Passed all diffDiag tests.")

Passed all diffRow tests.
Passed all diffDiag tests.


### The N-Queens CSP Class
Implement the CSP class as described above, with constraints to make sure each queen is on a different row and different diagonal than every other queen, and a variable for each column defining the row that containing a queen in that column.

In [ ]:
first_queen_row = symbols('first\_queen\_row')
second_queen_row = symbols('second\_queen\_row')
first_queen_col = symbols('first\_queen\_col')
second_queen_col = symbols('second\_queen\_col')

diffRow = constraint('diffRow', Ne(first_queen_row, second_queen_row))
diffDiag = constraint('diffDiag', Ne( abs(first_queen_row - second_queen_row), abs(first_queen_col - second_queen_col) ))

In [ ]:
class NQueensCSP:
    """CSP representation of the N-queens problem
    
    Parameters
    ----------
    N : Integer
        The side length of a square chess board to use for the problem, and
        the number of queens that must be placed on the board
    """
    def __init__(self, N):
        _vars = [symbols('queen_{}'.format(i)) for i in range(N)]
        _domain = set(range(N))
        self.size = N
        self.variables = _vars
        self.domains = {v: _domain for v in _vars}
        self._constraints = {queen: set() for queen in _vars}
        for queen in _vars:
            other_queens = [other_queen for other_queen in _vars if other_queen != queen]
            for other_queen in other_queens:
                
                self._constraints[queen].update(
                    set([
                        diffRow.subs({
                            first_queen_row: queen, 
                            second_queen_row: other_queen
                        }),
                        diffDiag.subs({
                            first_queen_row: queen, 
                            second_queen_row: other_queen,
                            first_queen_col: int(queen.name[-1]),
                            second_queen_col: int(other_queen.name[-1])
                        })
                    ])
                )

        # add constraints - for each pair of variables xi and xj, create
        # a diffRow(xi, xj) and a diffDiag(xi, xj) instance, and add them
        # to the self._constraints dictionary keyed to both xi and xj;
        # (i.e., add them to both self._constraints[xi] and self._constraints[xj])
    
    @property
    def constraints(self):
        """Read-only list of constraints -- cannot be used for evaluation """
        constraints = set()
        for _cons in self._constraints.values():
            constraints |= _cons
        return list(constraints)
    
    def is_complete(self, assignment):
        """An assignment is complete if it is consistent, and all constraints
        are satisfied.
        
        Hint: Backtracking search checks consistency of each assignment, so checking
        for completeness can be done very efficiently
        
        Parameters
        ----------
        assignment : dict(sympy.Symbol: Integer)
            An assignment of values to variables that have previously been checked
            for consistency with the CSP constraints
        """
        return set( assignment.values() ) == set( range(self.size) )
    
    def is_consistent(self, var, value, assignment):
        """Check consistency of a proposed variable assignment
                
        self._constraints[x] returns a set of constraints that involve variable `x`.
        An assignment is consistent unless the assignment it causes a constraint to
        return False (partial assignments are always consistent).
        
        Parameters
        ----------
        var : sympy.Symbol
            One of the symbolic variables in the CSP
            
        value : Numeric
            A valid value (i.e., in the domain of) the variable `var` for assignment

        assignment : dict(sympy.Symbol: Integer)
            A dictionary mapping CSP variables to row assignment of each queen
            
        """
        queen = var
        for other_queen in assignment:
            for constraint in self._constraints[queen]:
                if not constraint.subs({queen: value, other_queen: assignment[other_queen]}):
                    return False
        return True
        
    def inference(self, var, value):
        """Perform logical inference based on proposed variable assignment
        
        Returns an empty dictionary by default; function can be overridden to
        check arc-, path-, or k-consistency; returning None signals "failure".
        
        Parameters
        ----------
        var : sympy.Symbol
            One of the symbolic variables in the CSP
        
        value : Integer
            A valid value (i.e., in the domain of) the variable `var` for assignment
            
        Returns
        -------
        dict(sympy.Symbol: Integer) or None
            A partial set of values mapped to variables in the CSP based on inferred
            constraints from previous mappings, or None to indicate failure
        """
        # TODO (Optional): Implement this function based on AIMA discussion
        return {}
    
    def show(self, assignment):
        """Display a chessboard with queens drawn in the locations specified by an
        assignment
        
        Parameters
        ----------
        assignment : dict(sympy.Symbol: Integer)
            A dictionary mapping CSP variables to row assignment of each queen
            
        """
        locations = [(i, assignment[j]) for i, j in enumerate(self.variables)
                     if assignment.get(j, None) is not None]
        displayBoard(locations, self.size)

## III. Backtracking Search
Implement the [backtracking search](https://github.com/aimacode/aima-pseudocode/blob/master/md/Backtracking-Search.md) algorithm (required) and helper functions (optional) from the AIMA text.  

In [ ]:
def select(csp, assignment):
    """Choose an unassigned variable in a constraint satisfaction problem """
    # TODO (Optional): Implement a more sophisticated selection routine from AIMA
    for var in csp.variables:
        if var not in assignment:
            return var
    return None

def order_values(var, assignment, csp):
    """Select the order of the values in the domain of a variable for checking during search;
    the default is lexicographically.
    """
    # TODO (Optional): Implement a more sophisticated search ordering routine from AIMA
    return csp.domains[var]

def backtracking_search(csp):
    """Helper function used to initiate backtracking search """
    return backtrack({}, csp)

def backtrack(assignment, csp):
    """Perform backtracking search for a valid assignment to a CSP
    
    Parameters
    ----------
    assignment : dict(sympy.Symbol: Integer)
        An partial set of values mapped to variables in the CSP
        
    csp : CSP
        A problem encoded as a CSP. Interface should include csp.variables, csp.domains,
        csp.inference(), csp.is_consistent(), and csp.is_complete().
    
    Returns
    -------
    dict(sympy.Symbol: Integer) or None
        A partial set of values mapped to variables in the CSP, or None to indicate failure
    """
    if csp.is_complete(assignment):
        return assignment
    var = select(csp, assignment)
    
    for value in order_values(var, assignment, csp):
        if csp.is_consistent(var, value, assignment):
            assignment[var] = value
            inferences = csp.inference(var, value)
            if inferences is not None:
                assignment.update(inferences)
                result = backtrack(assignment, csp)
                if result:
                    return result
        assignment.pop(var, None)
    return None

### Solve the CSP
With backtracking implemented, now you can use it to solve instances of the problem. We've started with the classical 8-queen version, but you can try other sizes as well.  Boards larger than 12x12 may take some time to solve because sympy is slow in the way its being used here, and because the selection and value ordering methods haven't been implemented.  See if you can implement any of the techniques in the AIMA text to speed up the solver!

In [ ]:
num_queens = 11
csp = NQueensCSP(num_queens)
var = csp.variables[0]
print("CSP problems have variables, each variable has a domain, and the problem has a list of constraints.")
print("Showing the variables for the N-Queens CSP:")
display(csp.variables)
print("Showing domain for {}:".format(var))
display(csp.domains[var])
print("And showing the constraints for {}:".format(var))
display(csp._constraints[var])

print("Solving N-Queens CSP...")
assn = backtracking_search(csp)
if assn is not None:
    csp.show(assn)
    print("Solution found:\n{!s}".format(assn))
else:
    print("No solution found.")